## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-27-04-11-45 +0000,48hills,Under the Stars: Satya’s rock-tinged R&B hits ...,https://48hills.org/2026/01/under-the-stars-sa...
1,2026-01-27-04-06-00 +0000,wsj,Anta Sports Set to Become Puma’s Largest Share...,https://www.wsj.com/business/anta-sports-set-t...
2,2026-01-27-04-04-03 +0000,bbc,Immigration chief departing Minneapolis as Tru...,https://www.bbc.com/news/articles/cgrdlr70qg4o...
3,2026-01-27-04-00-00 +0000,wsj,"Stung by Trump, America’s Top Trading Partners...",https://www.wsj.com/world/us-trade-partners-ch...
4,2026-01-27-03-59-24 +0000,nyt,Here’s the latest.,https://www.nytimes.com/live/2026/01/26/us/min...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2407/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
20,trump,51
19,minneapolis,25
42,minnesota,21
99,ice,18
58,shooting,16


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
278,2026-01-26-12-03-00 +0000,wsj,Republican unity over President Trump’s immigr...,https://www.wsj.com/politics/policy/miller-noe...,135
37,2026-01-27-01-29-20 +0000,nypost,ICE tensions could be thawing in Minn. after T...,https://nypost.com/2026/01/26/us-news/ice-tens...,131
20,2026-01-27-03-01-00 +0000,wsj,"Democratic Rep. Ilhan Omar of Minnesota, facin...",https://www.wsj.com/politics/policy/minnesota-...,122
2,2026-01-27-04-04-03 +0000,bbc,Immigration chief departing Minneapolis as Tru...,https://www.bbc.com/news/articles/cgrdlr70qg4o...,119
89,2026-01-26-23-04-08 +0000,wapo,President Donald Trump said Monday that he spo...,https://www.washingtonpost.com,113


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
278,135,2026-01-26-12-03-00 +0000,wsj,Republican unity over President Trump’s immigr...,https://www.wsj.com/politics/policy/miller-noe...
12,113,2026-01-27-03-35-02 +0000,nypost,Alex Pretti was among 50 anti-ICE protesters w...,https://nypost.com/2026/01/26/us-news/alex-pre...
11,67,2026-01-27-03-36-00 +0000,wsj,"Deadly Winter Storm Disrupts Air Travel, Leave...",https://www.wsj.com/us-news/climate-environmen...
20,63,2026-01-27-03-01-00 +0000,wsj,"Democratic Rep. Ilhan Omar of Minnesota, facin...",https://www.wsj.com/politics/policy/minnesota-...
58,55,2026-01-27-00-42-00 +0000,wsj,The Army will gain access to Salesforce’s tech...,https://www.wsj.com/politics/national-security...
227,45,2026-01-26-16-07-27 +0000,nypost,Stock market grew in 2025 — but less than any ...,https://nypost.com/2026/01/26/business/stock-m...
133,43,2026-01-26-21-31-00 +0000,wsj,"Gold and Silver Surge to Records, Buoyed by Co...",https://www.wsj.com/finance/stocks/global-stoc...
56,39,2026-01-27-00-46-17 +0000,nypost,Citi ex-employee accuses top exec of ‘egregiou...,https://nypost.com/2026/01/26/business/citigro...
202,39,2026-01-26-17-54-24 +0000,nypost,At least 18 dead after ferry carrying over 350...,https://nypost.com/2026/01/26/world-news/at-le...
37,35,2026-01-27-01-29-20 +0000,nypost,ICE tensions could be thawing in Minn. after T...,https://nypost.com/2026/01/26/us-news/ice-tens...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
